In [114]:
import pandas as pd
import os
from sklearn.datasets import fetch_20newsgroups
import nltk
from sklearn.model_selection import GridSearchCV
import numpy as np
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from transformers import TFBertTokenizer
# from nlp import nlp
import preprocessor as p
import spacy
from torch.utils.data import DataLoader
import re
nlp = spacy.load('en_core_web_sm')
import torch.nn as nn
lemma = WordNetLemmatizer()
from sklearn.svm import SVC



from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample


import torch 
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
import fastText
import warnings
from tqdm import tqdm
import torch.nn as nn
warnings.filterwarnings("ignore")

import warnings
warnings.filterwarnings('ignore')

In [57]:
data = pd.read_csv("uncleaned_combined_txt.csv")

In [70]:
data

,label,text
0,0,My autistic 3 year old said her first sentence...
1,0,From having the left side of my lips almost co...
2,0,"After dating for just over two years, I propos..."
3,0,One day at a time. Smiling more makes a big di...
4,0,trying my hardest to be happy today. I hope my...
...,...,...
1759,1,death sound appealingthought death thing bring...
1760,1,left relationship almost 5 years depression en...
1761,1,always wanted friendlife never anyone loved ca...
1762,1,deal someone avoids everythinghi close friend ...


In [55]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['emoji_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['url_removed_data'] = 0
    df1['lemma_data'] = 0
    df1['spelling_checked_data'] = 0
    stop_words = set(stopwords.words("english"))

#   lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


  # iterate over each row of dataset and preprocess data
    for i in range(df1.shape[0]):

        # white space removel
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.text[i])
        emoji_removed = re.sub(r'\W+', ' ', df1['white_space_removed'][i].encode('ascii', 'ignore').decode('utf-8'), flags=re.UNICODE).strip()
        df1['emoji_removed'] = emoji_removed

        # lower casing and tokenization
        lower = df1['emoji_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)


        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)


        # remove urls and html tags
        urls = re.findall("https?://[a-zA-Z0-9_\?=\@\/#=.~-]+", " ".join(stopword_removed_data))
        url_removed_data = [x for x in stopword_removed_data if x not in urls]
        df1['url_removed_data'][i] = url_removed_data
        # print(url_removed_data)
        # print(type(url_removed_data))
        
        
        # punctuation removel
        punct_removed_data = [x for x in url_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)
        

        spelling_checked_data = [spell(x) for x in punct_removed_data]
        df1['spelling_checked_data'][i] = " ".join(spelling_checked_data)
#         print(" ".join(spelling_checked_data))

        lemma_data = [lemma.lemmatize(x) for x in spelling_checked_data]
        df1['lemma_data'][i] = lemma_data
        # spelling checking

    return df1

In [67]:
data.iloc[0]

label                                                    0
text     My autistic 3 year old said her first sentence...
Name: 0, dtype: object

In [68]:
preprocessed_data = preprocessing(data)

In [69]:
preprocessed_data

,label,text,white_space_removed,emoji_removed,tokenized_data,stopword_removed_data,punct_removed_data,url_removed_data,lemma_data,spelling_checked_data
0,0,My autistic 3 year old said her first sentence...,My autistic 3 year old said her first sentence...,turns might depressedhey became orphan cry hon...,"[my, autistic, 3, year, old, said, her, first,...","[autistic, 3, year, old, said, first, sentence...","[autistic, 3, year, old, said, first, sentence...","[autistic, 3, year, old, said, first, sentence...","[autistic, 3, year, old, said, first, sentence...",autistic 3 year old said first sentence today ...
1,0,From having the left side of my lips almost co...,From having the left side of my lips almost co...,turns might depressedhey became orphan cry hon...,"[from, having, the, left, side, of, my, lips, ...","[left, side, lips, almost, completely, torn, f...","[left, side, lips, almost, completely, torn, f...","[left, side, lips, almost, completely, torn, f...","[left, side, lip, almost, completely, torn, fa...",left side lips almost completely torn face dog...
2,0,"After dating for just over two years, I propos...","After dating for just over two years, I propos...",turns might depressedhey became orphan cry hon...,"[after, dating, for, just, over, two, years, i...","[dating, two, years, proposed, said, yes, enga...","[dating, two, years, proposed, said, yes, enga...","[dating, two, years, proposed, said, yes, enga...","[dating, two, year, proposed, said, yes, engaged]",dating two years proposed said yes engaged
3,0,One day at a time. Smiling more makes a big di...,One day at a time. Smiling more makes a big di...,turns might depressedhey became orphan cry hon...,"[one, day, at, a, time, smiling, more, makes, ...","[one, day, time, smiling, makes, big, difference]","[one, day, time, smiling, makes, big, difference]","[one, day, time, smiling, makes, big, difference]","[one, day, time, smiling, make, big, difference]",one day time smiling makes big difference
4,0,trying my hardest to be happy today. I hope my...,trying my hardest to be happy today. I hope my...,turns might depressedhey became orphan cry hon...,"[trying, my, hardest, to, be, happy, today, i,...","[trying, hardest, happy, today, hope, face, br...","[trying, hardest, happy, today, hope, face, br...","[trying, hardest, happy, today, hope, face, br...","[trying, hardest, happy, today, hope, face, br...",trying hardest happy today hope face brings so...
...,...,...,...,...,...,...,...,...,...,...
1759,1,death sound appealingthought death thing bring...,death sound appealingthought death thing bring...,turns might depressedhey became orphan cry hon...,"[death, sound, appealingthought, death, thing,...","[death, sound, appealingthought, death, thing,...","[death, sound, appealingthought, death, thing,...","[death, sound, appealingthought, death, thing,...","[death, sound, appealingthought, death, thing,...",death sound appealingthought death thing bring...
1760,1,left relationship almost 5 years depression en...,left relationship almost 5 years depression en...,turns might depressedhey became orphan cry hon...,"[left, relationship, almost, 5, years, depress...","[left, relationship, almost, 5, years, depress...","[left, relationship, almost, 5, years, depress...","[left, relationship, almost, 5, years, depress...","[left, relationship, almost, 5, year, depressi...",left relationship almost 5 years depression en...
1761,1,always wanted friendlife never anyone loved ca...,always wanted friendlife never anyone loved ca...,turns might depressedhey became orphan cry hon...,"[always, wanted, friendlife, never, anyone, lo...","[always, wanted, friendlife, never, anyone, lo...","[always, wanted, friendlife, never, anyone, lo...","[always, wanted, friendlife, never, anyone, lo...","[always, wanted, friendly, never, anyone, love...",always wanted friendlies never anyone loved ca...
1762,1,deal someone avoids everythinghi close friend ...,deal someone avoids everythinghi close

In [122]:
df = pd.DataFrame(columns = ['text', 'target'],index = None)
df['text'] = preprocessed_data['spelling_checked_data']
df['target'] = data['label']

In [124]:
df.to_csv("cleaned_data_all.csv", index = False)
cleaned_df = pd.read_csv("cleaned_data_all.csv")

In [85]:
path = "cleaned_data_all.csv"

In [87]:
non_cont_model = fastText.train_unsupervised(path, model='skipgram')

In [110]:
df_with_embedding = df.copy()
for i in range(len(df)):
    embeddings = np.array(non_cont_model.get_sentence_vector(df['text'][i]))
    df_with_embedding['text'][i] = embeddings
    df_with_embedding['target'][i] = df['target'][i]

In [118]:
df_with_embedding = df_with_embedding.rename(columns={'text':'Embeddings'})

In [116]:
df_with_embedding

,text,target
0,"[-0.039855443, 0.036473524, -0.20623395, 0.076...",0
1,"[-0.03649728, 0.033170026, -0.21308982, 0.0874...",0
2,"[-0.047610383, 0.030810263, -0.20189077, 0.061...",0
3,"[-0.03685748, 0.040161744, -0.21739762, 0.0895...",0
4,"[-0.027496582, 0.04259511, -0.22344425, 0.1079...",0
...,...,...
1759,"[-0.020344233, 0.038744725, -0.21521005, 0.109...",1
1760,"[-0.02823309, 0.035109952, -0.20898741, 0.0936...",1
1761,"[-0.026655467, 0.036016535, -0.20715354, 0.091...",1
1762,"[-0.027852943, 0.036258657, -0.21016794, 0.094...",1


In [254]:
df_with_embedding.to_csv("clean_data_with_embeddings.csv", index = False)
cleaned_df = pd.read_csv("clean_data_with_embeddings.csv")

In [130]:
train_x, test_x, train_y, test_y = train_test_split(df_with_embedding['Embeddings'], df_with_embedding['target'], train_size=0.2, random_state = 32 )

In [143]:
train_x.iloc[0]

array([-0.02511302,  0.03815467, -0.21342003,  0.10002109, -0.065419  ,
       -0.07697617,  0.09253636,  0.11956444, -0.00414546, -0.05217087,
       -0.14393789,  0.15659133, -0.03971355,  0.09986812, -0.04285796,
       -0.00771237, -0.08103644,  0.0842102 , -0.13466698, -0.08008885,
        0.06396492,  0.24965456, -0.17380159,  0.14163958,  0.0034283 ,
       -0.13874602, -0.12467469, -0.01913846,  0.03838994,  0.0362975 ,
       -0.01002271, -0.01973843, -0.10862789, -0.08652105,  0.00454351,
        0.04948609,  0.01412504,  0.01680061,  0.00942602, -0.22177272,
       -0.02632576, -0.02603904,  0.02229575, -0.07423657,  0.01040624,
       -0.09878636, -0.05566059,  0.01560424,  0.04104643,  0.04644802,
       -0.2168769 ,  0.14411113, -0.05867142,  0.09552859,  0.13457732,
        0.11857323, -0.03419296, -0.00096229,  0.10469908,  0.14624128,
        0.15130551,  0.13236019,  0.10106458, -0.01799941,  0.04789725,
        0.12564701, -0.01646347,  0.09134252,  0.09216394, -0.09

In [164]:
embeddings=[]
for i in range(len(train_x)):
    embeddings.append(train_x.iloc[i])
    
embeddings_test = []
for i in range(len(test_x)):
    embeddings_test.append(test_x.iloc[i])

In [162]:
model = SVC(kernel='rbf', C=10, gamma = 6)
model.fit(embeddings, train_y)

SVC(C=10, gamma=6)

In [163]:
model.score(embeddings_test,test_y)

0.8647308781869688

In [167]:
def form_data(data_frame):
    n_examples = len(data_frame)
    train_examples = []#store input example
    for i in range(n_examples):
        example = data_frame.loc[i]
        #sent_a  and sent_b with scores as label is added to train example
        train_examples.append(InputExample(texts=[example['text']],label=example['target'].astype('float32')))
    return train_examples

In [157]:
def get_model_predicts(data_type, trained_model):
    predicts = []
    for i in tqdm(range(len(data_type))):
        #convert into embeddings 
        embeddings1 = np.array([trained_model.encode(data_type.loc[i]['text'],convert_to_numpy=True)])
        #find cosine similarity
    return predicts

array([1, 1, 1, ..., 0, 0, 1], dtype=int64)

In [198]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [239]:
def preprocess_function(examples):
    preprocessed = []
    for i in range(len(examples)):
        preprocessed.append(tokenizer(examples['text'].iloc[i], truncation='longest_first'))
    examples
    return preprocessed

In [220]:
tokenizer(df['text'][0])


{'input_ids': [101, 8740, 16774, 2594, 1017, 2095, 2214, 2056, 2034, 6251, 2651, 2342, 8549, 2113, 2116, 2616, 2081, 5390, 2963, 2224, 2362, 2036, 3728, 2318, 10320, 2184, 2126, 2758, 20682, 1021, 15288, 3407, 5082, 2172, 19366, 2144, 4083, 10639, 12476, 2154, 3071, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [222]:
preprocessed = preprocess_function(df)

In [228]:
df[:1000]

,text,target
0,autistic 3 year old said first sentence today ...,0
1,left side lips almost completely torn face dog...,0
2,dating two years proposed said yes engaged,0
3,one day time smiling makes big difference,0
4,trying hardest happy today hope face brings so...,0
...,...,...
995,world moves withouttoday whim decided google s...,1
996,call helped 23 colorado struggled since kid ne...,1
997,wronghello know start try keep short clear und...,1
998,give completelymany years trying success decid...,1


In [223]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [224]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [225]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [245]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [253]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocess_function(df[:1000]),
    eval_dataset=preprocess_function(df[1000:]),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 1000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 126
  Number of trainable parameters = 66955010


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [200]:
tokenizer("akshara nair")

{'input_ids': [101, 17712, 7377, 2527, 22525, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [251]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\Ammu\.huggingface\token
Login successful


In [ ]:
imdb["test"][0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)